# Terrestrial Carbon Flux (TCF) Model Demo

In [ ]:
import numpy as np
from matplotlib import pyplot

%alias head head

Here's a map of the location of one of our fields.

![](assets/field_map.png)

And here's a satellite view.

![](assets/field_satellite.png)

## Quick Example

In [ ]:
from agstack.models import TCF
from agstack.drivers import drivers_for_tcf
from agstack.io import drivers_from_csv, params_dict_from_json

data, dates = drivers_from_csv('/home/arthur/Workspace/NTSG/projects/Y2023_Field-Scale_C_Flux/data/752731_field2_2D.csv')

# Load driver (metorology)
drivers = drivers_for_tcf(data)

# Soil organic carbon state from SoilGrids 250m
state = np.array([[150, 150, 450]]).T

# Read in model parameters
params = params_dict_from_json('../agstack/data/SPL4CMDL_V7_BPLUT.json')

model = TCF(params, [7], state)
model.spin_up(dates, drivers)

nee, gpp, rh = model.forward_run(drivers)

In [ ]:
pyplot.figure(figsize = (10, 5))
pyplot.plot(dates, nee[0])
pyplot.ylabel('Net Carbon Balance (g C m-2 day-1)')
pyplot.show()

## Step 1 of 3: Read in Data

In [ ]:
%head /home/arthur/Workspace/NTSG/projects/Y2023_Field-Scale_C_Flux/data/3e0165_field1_2D.csv

In [ ]:
base_dir = '/home/arthur/Workspace/NTSG/projects/Y2023_Field-Scale_C_Flux/data/'
field1, dates = drivers_from_csv(f'{base_dir}/3e0165_field1_2D.csv')
field2, _     = drivers_from_csv(f'{base_dir}/752731_field2_2D.csv')

# One field has a longer record than the other
field2 = np.concatenate([field2, np.nan * np.ones((9,60))], axis = 1)

# Create a (P x N x T) array for N fields, T time steps
data = np.stack([field1, field2], axis = 0).swapaxes(0, 1)
drivers = drivers_for_tcf(data)

In [ ]:
soc_state = np.array([150, 160]) # [g C m-2] in top 5 cm

# Starting guess for SOC content in three (3) pools
soc_state = soc_state * np.array([[1, 1, 3]]).T
soc_state

In [ ]:
land_cover = [7, 7]
params = params_dict_from_json('../agstack/data/SPL4CMDL_V7_BPLUT.json')

## Step 2 of 3: Spin-up Model

In [ ]:
model = TCF(params, land_cover, soc_state)
tolerance = model.spin_up(dates, drivers)

In [ ]:
pyplot.plot(tolerance.T)
pyplot.xlabel('Annual Cycles')
pyplot.ylabel('Change in Annual NEE Sum (g C m-2)')
pyplot.show()

In [ ]:
model.state.soc

In [ ]:
model.state.soc.sum(axis = 0)

## Step 3 of 3: Simulation!

In [ ]:
gpp = model.gpp(drivers[0:6])

pyplot.plot(dates, gpp[0])
pyplot.xlabel('Time (days)')
pyplot.ylabel('GPP (g C m-2 day-1)')
pyplot.show()

In [ ]:
nee, gpp, rh = model.forward_run(drivers)

In [ ]:
pyplot.figure(figsize = (10, 5))
pyplot.plot(dates, nee[0] - nee[1])
pyplot.ylabel('Diff. in Net Carbon Balance (g C m-2 day-1)')